# Lab 6: Build an ELT Pipeline with dbt on Postgres (Google Colab Version)

In this Colab we:

- Install and start **Postgres** inside the Colab VM.
- Install **dbt-core** and **dbt-postgres** via `pip`.
- Programmatically create the dbt **project** and **profiles** instead of using interactive prompts.
- Use Python and SQL from this notebook instead of pgAdmin.
- Generate dbt **documentation** and show where to download it.

> 💡 **Goal:** Experience an end-to-end ELT pipeline where raw data is loaded into Postgres, transformed with dbt, tested, and documented.


## Task 1 – Setting up dbt and Postgres

In this Colab notebook we:

- We install **Postgres** and start the database server inside the Colab VM.
- We install **dbt-core** and **dbt-postgres** with `pip`.
- We programmatically create the **dbt project** folder and `profiles.yml` file with the correct Postgres settings.
- Then we run `dbt debug` to verify connectivity.

Run the following cell **once** at the top of your session.


In [1]:
# Task 1 – Install Postgres and dbt, and start the Postgres service
# This may take a few minutes the first time it runs.

import os, time

# Install Postgres
!apt-get -y update
!apt-get -y install postgresql postgresql-contrib

# Start the PostgreSQL service
!service postgresql start

# Set a password for the 'postgres' superuser so we can connect via TCP
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"

# Install dbt-core and dbt-postgres
!pip install -q dbt-core dbt-postgres psycopg2-binary

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/u

### Connect to Postgres from Python

In the original lab, you work with Postgres via **pgAdmin**. In this notebook, we will:

1. Connect to the `postgres` database as the `postgres` superuser.
2. Keep a reusable connection and cursor.
3. Define a helper function `run_sql` to execute SQL commands and optionally print results.

> 🔁 If you restart the runtime, re-run the previous cell and this one to re-establish the connection.


In [2]:
import psycopg2

# Connect to the default 'postgres' database over TCP using the password we set above.
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port=5432,
)
conn.autocommit = True
cur = conn.cursor()

def run_sql(sql, fetch=False, max_rows=20):
    """Helper to run SQL against PostgreSQL and optionally print results.

    Args:
        sql (str): SQL command(s) to execute.
        fetch (bool): If True, fetch and print results.
        max_rows (int): Max number of rows to display when fetching.
    """
    print("Running SQL:\n" + "-" * 80)
    print(sql.strip())
    print("-" * 80)
    cur.execute(sql)

    if fetch:
        rows = cur.fetchall()
        print(f"Returned {len(rows)} rows (showing up to {max_rows}):")
        for row in rows[:max_rows]:
            print(row)
    else:
        try:
            print(f"Rowcount: {cur.rowcount}")
        except Exception:
            pass

    print("\n")

### Creating the dbt Project and Profile

In the original lab, `dbt init postgres_elt_pipeline` prompts you for:

- **Adapter**: postgres  
- **Host**: e.g., `sql-server` or IP  
- **Port**: `5432`  
- **User**: `postgres`  
- **Password**: `Passw0rd!`  
- **Database**: `postgres`  
- **Schema**: `public`  
- **Threads**: `4`  

Here in Colab we:

1. Create a folder `postgres_elt_pipeline` for the dbt project.
2. Create `dbt_project.yml` with basic project metadata.
3. Create `~/.dbt/profiles.yml` that dbt uses to connect to Postgres running in this notebook.
4. Use host `127.0.0.1`, user `postgres`, password `postgres` (set earlier), DB `postgres`, schema `public`.

Run the following cell to set this up.


In [3]:
import pathlib

project_dir = pathlib.Path("postgres_elt_pipeline")
models_dir = project_dir / "models"

# Create project and models directories
models_dir.mkdir(parents=True, exist_ok=True)

# dbt project configuration (equivalent to dbt init output)
dbt_project_yml = """name: 'postgres_elt_pipeline'
version: '1.0.0'
profile: 'postgres_elt_pipeline'

models:
  postgres_elt_pipeline:
    +schema: public
    +materialized: view
"""

with open(project_dir / "dbt_project.yml", "w") as f:
    f.write(dbt_project_yml)

# dbt profiles configuration under ~/.dbt/profiles.yml
profiles_dir = pathlib.Path.home() / ".dbt"
profiles_dir.mkdir(parents=True, exist_ok=True)

profiles_yml = """postgres_elt_pipeline:
  outputs:
    dev:
      type: postgres
      host: 127.0.0.1
      user: postgres
      password: postgres
      port: 5432
      dbname: postgres
      schema: public
      threads: 4
  target: dev
"""

with open(profiles_dir / "profiles.yml", "w") as f:
    f.write(profiles_yml)

print("Created dbt project at:", project_dir.resolve())
print("Created profiles.yml at:", profiles_dir / "profiles.yml")

Created dbt project at: /content/postgres_elt_pipeline
Created profiles.yml at: /root/.dbt/profiles.yml


### Verifying the dbt Connection (`dbt debug`)

In the original lab, you run:

```bash
dbt debug
```

to confirm that dbt can connect to Postgres using the settings in `profiles.yml`.

We do the same here, but make sure to run dbt from inside the `postgres_elt_pipeline` project directory.


In [4]:
# Run dbt debug from the project directory to test the connection
!cd postgres_elt_pipeline && dbt debug

18:33:10  Running with dbt=1.10.15
18:33:10  dbt version: 1.10.15
18:33:10  python version: 3.12.12
18:33:10  python path: /usr/bin/python3
18:33:10  os info: Linux-6.6.105+-x86_64-with-glibc2.35
18:33:10  Using profiles dir at /root/.dbt
18:33:10  Using profiles.yml file at /root/.dbt/profiles.yml
18:33:10  Using dbt_project.yml file at /content/postgres_elt_pipeline/dbt_project.yml
18:33:10  adapter type: postgres
18:33:10  adapter version: 1.9.1
18:33:10  Configuration:
18:33:10    profiles.yml file [OK found and valid]
18:33:10    dbt_project.yml file [OK found and valid]
18:33:10  Required dependencies:
18:33:10   - git [OK found]

18:33:10  Connection:
18:33:10    host: 127.0.0.1
18:33:10    port: 5432
18:33:10    user: postgres
18:33:10    database: postgres
18:33:10    schema: public
18:33:10    connect_timeout: 10
18:33:10    role: None
18:33:10    search_path: None
18:33:10    keepalives_idle: 0
18:33:10    sslmode: None
18:33:10    sslcert: None
18:33:10    sslkey: None
18:3

## Task 2 – Loading Raw Data into Postgres

In the original lab, you:

1. Opened **pgAdmin 4** and connected to the `postgres` database.
2. Created a table `public.raw_orders` using the Query Tool.
3. Inserted several sample rows (including some imperfect data that will be cleaned later).
4. Ran `SELECT * FROM raw_orders;` to verify.

In this notebook, we will execute the same SQL commands directly from Python using our `run_sql` helper.

### 2.1 Create the `raw_orders` Table

The table definition mirrors the lab:

```sql
CREATE TABLE public.raw_orders (
  order_id INT PRIMARY KEY,
  customer_name VARCHAR(100),
  product VARCHAR(100),
  quantity INT,
  price DECIMAL(10,2),
  order_date TIMESTAMP
);
```

Run the next cell to create the table.


In [5]:
create_table_sql = """
DROP TABLE IF EXISTS public.raw_orders;

CREATE TABLE public.raw_orders (
  order_id INT PRIMARY KEY,
  customer_name VARCHAR(100),
  product VARCHAR(100),
  quantity INT,
  price DECIMAL(10,2),
  order_date TIMESTAMP
);
"""

run_sql(create_table_sql)

Running SQL:
--------------------------------------------------------------------------------
DROP TABLE IF EXISTS public.raw_orders;

CREATE TABLE public.raw_orders (
  order_id INT PRIMARY KEY,
  customer_name VARCHAR(100),
  product VARCHAR(100),
  quantity INT,
  price DECIMAL(10,2),
  order_date TIMESTAMP
);
--------------------------------------------------------------------------------
Rowcount: -1




### 2.2 Insert Sample Raw Data

The original lab inserts a small set of orders, including:

- Valid orders (e.g., Alice buying a Laptop).
- Some **dirty data**, like:
  - A row with **empty product** and **zero quantity**.
  - Rows where `quantity` is provided as a string.

We insert the same data here (with a tiny syntax fix: adding a missing comma between rows 4 and 5).

Run the next cell to insert the sample data.


In [6]:
insert_data_sql = """
INSERT INTO public.raw_orders (order_id, customer_name, product, quantity, price, order_date) VALUES
(1, 'Alice',   'Laptop',     1, 1200.00, '2024-02-01 10:15:00'),
(2, 'Bob',     'Mouse',      2,   25.00, '2024-02-02 12:30:00'),
(3, 'Charlie', 'Keyboard',   1,   50.00, '2024-02-03 14:45:00'),
(4, 'David',   'Monitor',    1,  300.00, '2024-02-04 16:00:00'),
(5, 'Lewis',   '',           0,    0.00, '2024-09-11 14:23:30'),
(6, 'Sally',   'Server',     3, 2100.00, '2024-09-11 14:14:22'),
(7, 'Claire',  'Disk Drive', 2,  172.00, '2024-09-23 11:23:15');
"""

run_sql(insert_data_sql)

Running SQL:
--------------------------------------------------------------------------------
INSERT INTO public.raw_orders (order_id, customer_name, product, quantity, price, order_date) VALUES
(1, 'Alice',   'Laptop',     1, 1200.00, '2024-02-01 10:15:00'),
(2, 'Bob',     'Mouse',      2,   25.00, '2024-02-02 12:30:00'),
(3, 'Charlie', 'Keyboard',   1,   50.00, '2024-02-03 14:45:00'),
(4, 'David',   'Monitor',    1,  300.00, '2024-02-04 16:00:00'),
(5, 'Lewis',   '',           0,    0.00, '2024-09-11 14:23:30'),
(6, 'Sally',   'Server',     3, 2100.00, '2024-09-11 14:14:22'),
(7, 'Claire',  'Disk Drive', 2,  172.00, '2024-09-23 11:23:15');
--------------------------------------------------------------------------------
Rowcount: 7




### 2.3 Verify the Loaded Raw Data

Just like running `SELECT * FROM raw_orders;` in pgAdmin, we’ll query the table here to confirm the data.


In [7]:
run_sql("SELECT * FROM public.raw_orders;", fetch=True, max_rows=10)

Running SQL:
--------------------------------------------------------------------------------
SELECT * FROM public.raw_orders;
--------------------------------------------------------------------------------
Returned 7 rows (showing up to 10):
(1, 'Alice', 'Laptop', 1, Decimal('1200.00'), datetime.datetime(2024, 2, 1, 10, 15))
(2, 'Bob', 'Mouse', 2, Decimal('25.00'), datetime.datetime(2024, 2, 2, 12, 30))
(3, 'Charlie', 'Keyboard', 1, Decimal('50.00'), datetime.datetime(2024, 2, 3, 14, 45))
(4, 'David', 'Monitor', 1, Decimal('300.00'), datetime.datetime(2024, 2, 4, 16, 0))
(5, 'Lewis', '', 0, Decimal('0.00'), datetime.datetime(2024, 9, 11, 14, 23, 30))
(6, 'Sally', 'Server', 3, Decimal('2100.00'), datetime.datetime(2024, 9, 11, 14, 14, 22))
(7, 'Claire', 'Disk Drive', 2, Decimal('172.00'), datetime.datetime(2024, 9, 23, 11, 23, 15))




## Task 3 – Defining and Executing dbt Transformations

In the original lab, you create a dbt model file:

`C:\\Users\\student\\postgres_elt_pipeline\\models\\clean_orders.sql`

with the following logic:

- Select from `{{ source('public', 'raw_orders') }}`.
- Clean customer names to lower-case.
- Compute a new `total_price` as `quantity * price`.
- Strip the time component from `order_date` and cast to `DATE`.
- Filter out rows where `quantity <= 0`.

### 3.1 Create `models/clean_orders.sql`

We’ll now create the same model file inside our project folder.


In [8]:
from pathlib import Path

clean_model_sql = """WITH cleaned AS (
  SELECT
    order_id,
    LOWER(customer_name) AS customer_name,
    product,
    quantity,
    price,
    quantity * price AS total_price,
    order_date::DATE AS order_date
  FROM {{ source('public', 'raw_orders') }}
  WHERE quantity > 0
)
SELECT * FROM cleaned
"""

clean_model_path = Path("postgres_elt_pipeline") / "models" / "clean_orders.sql"
with open(clean_model_path, "w") as f:
    f.write(clean_model_sql)

print("Wrote clean_orders model to:", clean_model_path.resolve())

Wrote clean_orders model to: /content/postgres_elt_pipeline/models/clean_orders.sql


## Task 4 – Verifying Transformations and Testing Data Integrity

In the original lab, you:

1. Create `models/schema.yml` and define the **source** `public.raw_orders`.
2. Delete contents of the `models/example` folder.
3. Run `dbt run` to build the `clean_orders` view in Postgres.
4. Confirm the view exists via pgAdmin.
5. Extend `schema.yml` with **tests** (unique, not_null) for `clean_orders` columns.
6. Run `dbt test` to validate data integrity.

We’ll follow the same structure here.

### 4.1 Define the Source in `models/schema.yml`

First, we declare `public.raw_orders` as a dbt **source**, so that `{{ source('public', 'raw_orders') }}` in `clean_orders.sql` is recognized.


In [9]:
schema_yml_sources = """version: 2

sources:
  - name: public
    database: postgres
    tables:
      - name: raw_orders
"""

from pathlib import Path
schema_path = Path("postgres_elt_pipeline") / "models" / "schema.yml"
with open(schema_path, "w") as f:
    f.write(schema_yml_sources)

print("Wrote initial schema.yml with sources to:", schema_path.resolve())

Wrote initial schema.yml with sources to: /content/postgres_elt_pipeline/models/schema.yml


### 4.2 Remove Example Models (if present)

The original lab asks you to delete the contents of the `models/example` folder so only your models remain.

Our manually-created project does not include example models by default, but we’ll still run a cleanup command that removes an `example` folder if it exists.


In [10]:
# Remove the example models folder if it exists (safe no-op otherwise)
!rm -rf postgres_elt_pipeline/models/example
print("Example models (if any) have been removed.")

Example models (if any) have been removed.


### 4.3 Run dbt Transformations (`dbt run`)

Now we run:

```bash
dbt run
```

from within the project directory. This should create a **view** named `clean_orders` in the `public` schema, based on our model.


In [11]:
# Run dbt to build the clean_orders view
!cd postgres_elt_pipeline && dbt run

18:34:44  Running with dbt=1.10.15
18:34:44  Registered adapter: postgres=1.9.1
18:34:45  Unable to do partial parsing because saved manifest not found. Starting full parse.
18:34:46  Found 1 model, 1 source, 459 macros
18:34:46  
18:34:46  Concurrency: 4 threads (target='dev')
18:34:46  
18:34:47  1 of 1 START sql view model public_public.clean_orders ......................... [RUN]
18:34:47  1 of 1 OK created sql view model public_public.clean_orders .................... [CREATE VIEW in 0.15s]
18:34:47  
18:34:47  Finished running 1 view model in 0 hours 0 minutes and 0.39 seconds (0.39s).
18:34:47  
18:34:47  Completed successfully
18:34:47  
18:34:47  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 NO-OP=0 TOTAL=1


### 4.4 Verify the `clean_orders` View in Postgres

Instead of checking pgAdmin, we query the view directly from this notebook.  
You should see:

- `customer_name` in lower-case.
- `total_price` as `quantity * price`.
- `order_date` with the time stripped off (DATE only).
- Rows with `quantity <= 0` removed.


In [13]:
run_sql("SELECT * FROM public_public.clean_orders ORDER BY order_id;", fetch=True, max_rows=20)

Running SQL:
--------------------------------------------------------------------------------
SELECT * FROM public_public.clean_orders ORDER BY order_id;
--------------------------------------------------------------------------------
Returned 6 rows (showing up to 20):
(1, 'alice', 'Laptop', 1, Decimal('1200.00'), Decimal('1200.00'), datetime.date(2024, 2, 1))
(2, 'bob', 'Mouse', 2, Decimal('25.00'), Decimal('50.00'), datetime.date(2024, 2, 2))
(3, 'charlie', 'Keyboard', 1, Decimal('50.00'), Decimal('50.00'), datetime.date(2024, 2, 3))
(4, 'david', 'Monitor', 1, Decimal('300.00'), Decimal('300.00'), datetime.date(2024, 2, 4))
(6, 'sally', 'Server', 3, Decimal('2100.00'), Decimal('6300.00'), datetime.date(2024, 9, 11))
(7, 'claire', 'Disk Drive', 2, Decimal('172.00'), Decimal('344.00'), datetime.date(2024, 9, 23))




### 4.5 Add Tests to `schema.yml`

The original lab extends `schema.yml` with a `models:` section to add:

- A **unique** and **not_null** test on `order_id`.
- A **not_null** test on `quantity`.

We’ll now update `models/schema.yml` to include both the **sources** and the **model tests**.


In [14]:
schema_yml_full = """version: 2

sources:
  - name: public
    database: postgres
    tables:
      - name: raw_orders

models:
  - name: clean_orders
    description: "Cleaned customer order data"
    columns:
      - name: order_id
        tests:
          - unique
          - not_null
      - name: quantity
        tests:
          - not_null
"""

with open(schema_path, "w") as f:
    f.write(schema_yml_full)

print("Updated schema.yml with model tests at:", schema_path.resolve())

Updated schema.yml with model tests at: /content/postgres_elt_pipeline/models/schema.yml


### 4.6 Run dbt Tests (`dbt test`)

Now we run:

```bash
dbt test
```

This executes the tests defined in `schema.yml`, checking:

- `clean_orders.order_id` is **unique** and **not null**.
- `clean_orders.quantity` is **not null**.


In [15]:
# Run dbt tests
!cd postgres_elt_pipeline && dbt test

18:36:20  Running with dbt=1.10.15
18:36:20  Registered adapter: postgres=1.9.1
18:36:21  Found 1 model, 3 data tests, 1 source, 459 macros
18:36:21  
18:36:21  Concurrency: 4 threads (target='dev')
18:36:21  
18:36:21  1 of 3 START test not_null_clean_orders_order_id ............................... [RUN]
18:36:21  2 of 3 START test not_null_clean_orders_quantity ............................... [RUN]
18:36:21  3 of 3 START test unique_clean_orders_order_id ................................. [RUN]
18:36:21  3 of 3 PASS unique_clean_orders_order_id ....................................... [PASS in 0.17s]
18:36:21  2 of 3 PASS not_null_clean_orders_quantity ..................................... [PASS in 0.17s]
18:36:21  1 of 3 PASS not_null_clean_orders_order_id ..................................... [PASS in 0.18s]
18:36:21  
18:36:21  Finished running 3 data tests in 0 hours 0 minutes and 0.34 seconds (0.34s).
18:36:21  
18:36:21  Completed successfully
18:36:21  
18:36:21  Done. PASS=3 WA

## Task 5 – Documenting and Reviewing Changes

In the original lab, you:

1. Run `dbt docs generate` to build documentation.
2. Run `dbt docs serve` to start a local web server and view docs in the browser.
3. Navigate to `clean_orders` in the **Database** tab to inspect the model and SQL.

In this Colab environment, we will:

- Run `dbt docs generate` to build the docs into the `target/` folder.
- Inspect the generated files.
- (Optionally) download the `target` folder or `index.html` to view locally.


In [16]:
# Generate dbt docs
!cd postgres_elt_pipeline && dbt docs generate

18:36:35  Running with dbt=1.10.15
18:36:35  Registered adapter: postgres=1.9.1
18:36:36  Found 1 model, 3 data tests, 1 source, 459 macros
18:36:36  
18:36:36  Concurrency: 4 threads (target='dev')
18:36:36  
18:36:36  Building catalog
18:36:36  Catalog written to /content/postgres_elt_pipeline/target/catalog.json


### 5.1 Inspect Generated Docs

Locally, you would next run `dbt docs serve` to launch a browser-based UI.  
Running a long-lived web server process is not ideal in Colab, so instead we:

- List the generated files in the `target` directory.
- Note that you can **download** `target/index.html` and open it in your browser to view the docs UI.


In [20]:
# List the generated documentation files
!ls -R postgres_elt_pipeline/target

print("\nTo view the documentation UI, download 'postgres_elt_pipeline/target/index.html' and open it in a local browser.")

postgres_elt_pipeline/target:
catalog.json	    index.html		   run_results.json
compiled	    manifest.json	   semantic_manifest.json
graph.gpickle	    partial_parse.msgpack
graph_summary.json  run

postgres_elt_pipeline/target/compiled:
postgres_elt_pipeline

postgres_elt_pipeline/target/compiled/postgres_elt_pipeline:
models

postgres_elt_pipeline/target/compiled/postgres_elt_pipeline/models:
clean_orders.sql  schema.yml

postgres_elt_pipeline/target/compiled/postgres_elt_pipeline/models/schema.yml:
not_null_clean_orders_order_id.sql  unique_clean_orders_order_id.sql
not_null_clean_orders_quantity.sql

postgres_elt_pipeline/target/run:
postgres_elt_pipeline

postgres_elt_pipeline/target/run/postgres_elt_pipeline:
models

postgres_elt_pipeline/target/run/postgres_elt_pipeline/models:
clean_orders.sql  schema.yml

postgres_elt_pipeline/target/run/postgres_elt_pipeline/models/schema.yml:
not_null_clean_orders_order_id.sql  unique_clean_orders_order_id.sql
not_null_clean_orders_quantity.sq

### 5.2 Query `clean_orders` One More Time

As a final verification (similar to the last step in the lab where you query `clean_orders` from pgAdmin), we run:

```sql
SELECT * FROM clean_orders;
```

to confirm:

- `total_price` reflects `quantity * price`.
- `order_date` is a `DATE` (no time component).
- `customer_name` is lower-case.


In [22]:
run_sql("SELECT * FROM public_public.clean_orders ORDER BY order_id;", fetch=True, max_rows=20)

Running SQL:
--------------------------------------------------------------------------------
SELECT * FROM public_public.clean_orders ORDER BY order_id;
--------------------------------------------------------------------------------
Returned 6 rows (showing up to 20):
(1, 'alice', 'Laptop', 1, Decimal('1200.00'), Decimal('1200.00'), datetime.date(2024, 2, 1))
(2, 'bob', 'Mouse', 2, Decimal('25.00'), Decimal('50.00'), datetime.date(2024, 2, 2))
(3, 'charlie', 'Keyboard', 1, Decimal('50.00'), Decimal('50.00'), datetime.date(2024, 2, 3))
(4, 'david', 'Monitor', 1, Decimal('300.00'), Decimal('300.00'), datetime.date(2024, 2, 4))
(6, 'sally', 'Server', 3, Decimal('2100.00'), Decimal('6300.00'), datetime.date(2024, 9, 11))
(7, 'claire', 'Disk Drive', 2, Decimal('172.00'), Decimal('344.00'), datetime.date(2024, 9, 23))




## Wrap-Up and Cleanup

You have now:

- Set up Postgres and dbt in a Colab environment.
- Loaded raw order data into Postgres.
- Created a dbt model to clean and transform the data.
- Added tests to validate data integrity.
- Generated dbt documentation for the project.

The optional cell below closes the Postgres connection in Python.


In [23]:
# Optional: close the database connection when you are done
cur.close()
conn.close()
print("PostgreSQL connection closed.")

PostgreSQL connection closed.
